# Q2


*   In this question we will Build a bigram HMM tagger.
*   First we split the part-of-speech-tagged corpus into a training set and test set.
*   From the labeled training set, we train the transition and observation probabilities of the HMM tagger directly on the hand-tagged data.
*   Then implement the Viterbi algorithm so we can decode a test sentence.
*   Run the algorithm on the test set. Report its error rate and
compare its performance to the most frequent tag baseline.
*   Build a confusion matrix and investigate the most frequent errors.


In [1]:
import nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [2]:
import math
import nltk
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [3]:
def collect_probabilities(_samples):
    """
    Collects various probabilities and frequencies required for Hidden Markov Model (HMM) training based on given samples.

    Args:
    _samples (list): A list of tuples containing (word, tag) pairs representing training data.

    Returns:
    tuple: A tuple containing four dictionaries:
           - tag_freq: A dictionary storing the frequency of each tag type.
           - word_per_tag_freq: A nested dictionary storing the frequency of each word under each tag.
           - bigram: A nested dictionary representing the bigram probabilities between tag types.
           - pi: A dictionary storing the initial probability distributions for tag types.

    Comments:
    - tag_freq: Stores the frequency of each tag type observed in the training data.
    - word_per_tag_freq: Stores the frequency of each word observed under each tag in the training data.
      - Each word under a specific tag is initialized with a count of 2 to provide initial smoothing.
    - bigram: Represents the transition probabilities between tag types based on bigram counts.
    - pi: Stores the initial probability distributions for tag types based on the second tag in each sample.
    """
    tag_freq = {}
    word_per_tag_freq = {}
    bigram = {}
    pi = {}
    samples_len = len(_samples)

    # Iterate through samples to collect tag frequencies, word frequencies under each tag, and initial probabilities
    for i in range(samples_len):
        sample = _samples[i]
        has_next = i + 1 < samples_len

        # Collect data for initial probabilities (pi)
        if has_next:
            if _samples[i + 1][1] not in pi:
                pi.update({_samples[i + 1][1]: 2})
            else:
               # Your code here
               pi[_samples[i + 1][1]] += 1  # Increment the count of the next tag in pi


        # Count tag frequencies (tag_freq) and word frequencies under each tag (word_per_tag_freq)
        if sample[1] not in tag_freq:
            tag_freq.update({sample[1]: 2})
            word_per_tag_freq.update({sample[1]: {sample[0]: 2}})
        else:
           # Your code here
           tag_freq[sample[1]] += 1  # Increment tag frequency
           if sample[0] not in word_per_tag_freq[sample[1]]:
               word_per_tag_freq[sample[1]][sample[0]] = 2  # Initialize with smoothing if word is new for the tag
           else:
               word_per_tag_freq[sample[1]][sample[0]] += 1  # Increment word frequency under the tag


    # Initialize the bigram matrix with default counts
    for tag_0 in tag_freq:
        bigram.update({tag_0: {}})
        for tag_1 in tag_freq:
            bigram[tag_0].update({tag_1: 1})

    # Count bigram occurrences
    for i in range(samples_len):
      # Your code here
      if i + 1 < samples_len:  # Check if the next index is within the range
          current_tag = _samples[i][1]
          next_tag = _samples[i + 1][1]
          if current_tag not in bigram:
              bigram[current_tag] = {}
          if next_tag not in bigram[current_tag]:
              bigram[current_tag][next_tag] = 2  # Initialize with 2 for smoothing and to account for unseen transitions
          else:
              bigram[current_tag][next_tag] += 1  # Increment the count of transition from current_tag to next_tag

    # Return collected probabilities and frequencies
    return tag_freq, word_per_tag_freq, bigram, pi


In [4]:
def create_confusion_matrix(predictions, hidden_state, confusion_matrices , is_correct):
    """
    Updates the confusion matrices based on the predictions and hidden states,
    calculates the number of correct predictions, and identifies tags with
    the most false positives and false negatives.

    Args:
    predictions (list): Predicted sequence of tags for the given input data.
    hidden_state (list): Actual sequence of tags for the given input data.
    confusion_matrices (dict): A dictionary containing confusion matrices for each tag.
    is_correct (int): Counter for the number of correct predictions.

    Returns:
    int: Updated count of correct predictions after processing the input data.

    Comments:
    - As the Viterbi algorithm processes each sentence in the test data,
      it compares the predicted tags against the actual tags for each word in the sentence.
        - For each word in the sentence, if the predicted tag matches the actual tag (a true positive),
          the corresponding count in the confusion matrix for that tag is incremented.
        - If the predicted tag does not match the actual tag:
            - If the predicted tag is incorrect (a false positive), the FP count for
              the predicted tag is incremented, and the TN counts for other tags are incremented.
            - If the actual tag is not predicted (a false negative), the FN count for the actual
              tag is incremented, and the TN counts for other tags are incremented.
    - After updating the confusion matrices, the function identifies tags with
      the highest counts of false positives and false negatives.
    """
    # fill in the confusion matrix
    for i in range(len(predictions)):
        if predictions[i] == hidden_state[i]:
            # prediction is correct
            is_correct += 1
            confusion_matrices[predictions[i]]['TP'] += 1  # Increment true positive count
            for j in confusion_matrices:
                # Your code here
                if j != predictions[i]:
                  confusion_matrices[j]['TN'] += 1  # Increment true negative count for other tags


        else:
            # prediction is incorrect
            confusion_matrices[predictions[i]]['FP'] += 1
            confusion_matrices[hidden_state[i]]['FN'] += 1  # Increment false negative count for actual tag
            for j in confusion_matrices:
                # Your code here
                if j != predictions[i] and j != hidden_state[i]:
                    confusion_matrices[j]['TN'] += 1  # Increment true negative for other tags


    # rank
    highest_FP_tag = []
    highest_FP = -1
    highest_FN_tag = []
    highest_FN = -1

    for tag in confusion_matrices:

        print(f'{tag}: {confusion_matrices[tag]}')

        if confusion_matrices[tag]['FP'] > highest_FP:
            highest_FP = confusion_matrices[tag]['FP']
            highest_FP_tag = tag

        if confusion_matrices[tag]['FN'] > highest_FN:
            highest_FN = confusion_matrices[tag]['FN']
            highest_FN_tag = tag

    print(f'\nTag with the most false positives is: {highest_FP_tag} with {highest_FP} counts.')
    print(f'Tag with the most false negative is:  {highest_FN_tag} with {highest_FN} counts.')

    return is_correct


In [5]:
def viterbi(_samples, _tag_freq, _word_per_tag_freq, _bigram, _init_dist):
    """
    Performs the Viterbi algorithm for part-of-speech tagging on a given test set of sentences.

    Args:
    _samples (list): A list of tuples containing (word, tag) pairs representing the test data.
    _tag_freq (dict): A dictionary storing the frequency of each tag type observed in the training data.
    _word_per_tag_freq (dict): A nested dictionary storing the frequency of each word observed under each tag in the training data.
    _bigram (dict): A nested dictionary representing the bigram probabilities between tag types.
    _init_dist (dict): A dictionary storing the initial probability distributions for tag types.

    Comments:
    - This function implements the Viterbi algorithm for part-of-speech tagging.
    - It processes each sentence in the test set, using '.' as an indicator that a sentence is over.
    - The function populates a confusion matrix to track the performance of the model.
    - The Viterbi algorithm is used to find the most likely sequence of tags for each sentence.
    - The algorithm iterates through each word in the sentence, calculating the most likely tag sequence based on probabilities and transition probabilities.
    - The model's predictions are compared against the actual tags to evaluate accuracy and populate the confusion matrix.
    - The function prints detailed information about predictions, including the sentence, hidden states, and predicted tags.
    - It also identifies tags with the most false positives and false negatives based on the confusion matrix.
    - Finally, the function prints the overall accuracy of the model on the test set.
    """
    test_size = len(_samples)
    current_index = 0
    is_correct = 0

    # populate the confusion matrix
    confusion_matrices = {}
    for tag in _tag_freq:
        confusion_matrices.update({tag: {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}})

    while current_index < test_size:
        # find the boundry of a sentence
        sentence = []
        hidden_state = []
        last_token = ''
        while last_token != '.' and current_index < test_size:
            # sentence.append(''' Your code here ''')
            # hidden_state.append(''' Your code here ''')
            sentence.append(_samples[current_index][0])
            hidden_state.append(_samples[current_index][1])
            last_token = sentence[-1]
            current_index += 1

        # initialization step
        path_probability = {}
        backpointer = {}
        for tag in init_dist:
            path_probability.update({tag: []})
            backpointer.update({tag: [0]})

            # initial distribution of this tag
            pi_tag = init_dist[tag]

            # b_word is the probability of the word being generated by this tag
            if sentence[0] in _word_per_tag_freq[tag]:
                # Your code here
                b_word = _word_per_tag_freq[tag][sentence[0]] / _tag_freq[tag]
            else:
                b_word = 2.2250738585072014e-100
            # path_probability[tag].append(''' Your code here ''')
            path_probability[tag].append(math.log(pi_tag * b_word, 10))

        # recursion step
        T = len(sentence)
        for i in range(1, T):
            for tag in init_dist:

                if sentence[i] in _word_per_tag_freq[tag]:
                    # Your code here
                    b_word = _word_per_tag_freq[tag][sentence[i]] / _tag_freq[tag]
                else:
                    b_word = 2.2250738585072014e-100

                # search the maximum value of
                # viterbi[s', t-1] * a(s|s') *b_s(o_t)
                best_trans_prob = -2.2250738585072014e+308
                best_trans_tag = ''
                for prev_tag in init_dist:
                    if prev_tag in _bigram and tag in _bigram[prev_tag]:
                        # transitional_prob = # Your code here
                        transitional_prob = _bigram[prev_tag][tag] / _tag_freq[prev_tag]
                    else:
                        transitional_prob = 2.2250738585072014e-100

                    prob = path_probability[prev_tag][i - 1] + math.log(transitional_prob * b_word, 10)

                    if prob > best_trans_prob:
                        best_trans_prob = prob
                        best_trans_tag = prev_tag

                path_probability[tag].append(best_trans_prob)
                backpointer[tag].append(best_trans_tag)

        # termination step
        best_path_prob = -2.2250738585072014e+308
        best_path_pointer = None
        for tag in init_dist:
            # Your code here
            if path_probability[tag][T - 1] > best_path_prob:
                best_path_prob = path_probability[tag][T - 1]
                best_path_pointer = tag

        predictions = [best_path_pointer]

        # make the predictions path
        for i in reversed(range(1, T)):
            # Your code here
            prev_tag = predictions[-1]
            predictions.append(backpointer[prev_tag][i])
        predictions.reverse()

        print(f'sentence:       {sentence}')
        print(f'hidden s:       {hidden_state}')
        print(f'predictions:    {predictions}')

        is_correct = create_confusion_matrix(predictions, hidden_state, confusion_matrices, is_correct)

    print(f'\nmodel got {is_correct} samples correct out of {test_size}')
    print(f'accuracy: {is_correct / test_size}')



In [6]:
CORPUS = brown.tagged_words(categories='news', tagset='universal')
CORPUS_SIZE = len(brown.tagged_words(categories='news'))

CUT_OFF = math.floor(CORPUS_SIZE * 0.75)

# section off training and testing lists from corpus
training_list = CORPUS[:CUT_OFF]
testing_list = CORPUS[CUT_OFF:]

tag_frequency, word_per_tag_frequency, tag_bigram, init_dist = collect_probabilities(training_list)
viterbi(testing_list, tag_frequency, word_per_tag_frequency, tag_bigram, init_dist)


Streaming output truncated to the last 5000 lines.
ADP: {'TP': 2251, 'FP': 223, 'TN': 16201, 'FN': 84}
.: {'TP': 2272, 'FP': 52, 'TN': 16435, 'FN': 0}
ADV: {'TP': 657, 'FP': 65, 'TN': 17861, 'FN': 176}
CONJ: {'TP': 565, 'FP': 4, 'TN': 18183, 'FN': 7}
PRT: {'TP': 419, 'FP': 37, 'TN': 18234, 'FN': 69}
PRON: {'TP': 524, 'FP': 35, 'TN': 18193, 'FN': 7}
NUM: {'TP': 301, 'FP': 6, 'TN': 18394, 'FN': 58}
X: {'TP': 13, 'FP': 1, 'TN': 18723, 'FN': 22}

Tag with the most false positives is: NOUN with 644 counts.
Tag with the most false negative is:  ADJ with 474 counts.
sentence:       ['Crime', ':', "'", 'skyjacked', "'", 'From', 'International', 'Airport', 'in', 'Los', 'Angeles', 'to', 'International', 'Airport', 'in', 'Houston', ',', 'as', 'the', 'great', 'four-jet', 'Boeing', '707', 'flies', ',', 'is', 'a', 'routine', 'five', 'hours', 'and', '25', 'minutes', ',', 'including', 'stopovers', 'at', 'Phoenix', ',', 'El', 'Paso', ',', 'and', 'San', 'Antonio', '.']
hidden s:       ['NOUN', '.', '.',